# TIMESERIES TO IMAGE CONVERTER


In [2]:
from nilmtk import DataSet
import numpy as np

from utils.log import print_parameters, print_progress, print_log, print_break, print_end, print_end_of_loop
from utils.init import get_appliances, param_setup
from utils.file_handling import store_many_hdf5, create_file
from utils.data_handling import mount_data, append_images
from utils.filters import filter_empty_slices_and_fill_missing_samples, filter_low_entropy_slices
from utils.process import transform_ts, moving_window

dataset_name = "iawe"
dataset = DataSet('datasets/'+dataset_name+'.h5')

In [3]:
par = {
    'step_in_mins': 13, # Window size
    'max_images': 10, # Number of images per appliance per building.
    'img_size': 100, # Output image size.
    'frames': 5, # Video frames
    'allowed_delta_between_frames': 600*3, # Allowed time difference between frames.
    'resample_period': 6, # Resamples time series data to given resample period. Sample period of dataset can be found in dataset.metadata
    'fill_limit': 10, # Limit of how many samples to backfill when resampling, larger back fill should yield larger output. 
    'percentage_of_missing_data_allowed': 0.85, # Missing data will be backfilled.
    'add_brightness': True, # Multiply non-zero mean power to image.
    'ts_save': False, # Save source time series.
    'trs_type': 'GAF', # GAF or RECU - Gramian Angular Field or Recurrance plot.
    'trs_type_gaf': 'GASF', # GASF or GADF
    'multiple_buildings': True, # If false it processes only one building.
    'selected_building': 1, # Used if parameter above is False.
    'manually_select_appliances': False # Can be set in get_appliances() function.
    }

In [4]:
# Fixes possible misconfiguration and obtains metadata
param_setup(dataset,par)

par["appliances"] = get_appliances(dataset,par)
print_parameters(par)
file_name = create_file(par)

# Define global metrics.
healthy_appliances = set()
images_stacked = 0 
images_stacked_per_appliance = 0 

# Collect at least "max_dataset_size" images for each appliance for every building.
for appliance in par["appliances"]:
    print_log(par,"\n","Starting "f"{appliance} ("f"{par['appliances'].index(appliance)+1}/"f"{len(par['appliances'])}):")

    # Define metric.
    images_stacked_per_appliance = 0
    
    # Loop through all buildings.
    for building in dataset.buildings:
        print_log(par,"\n","Starting building "f"{building}")

        # Define temporary array to store image / frames. 
        img_stack_tmp = np.zeros([0, par["img_size"], par["img_size"]])
        sig_stack_tmp = np.zeros([0, par["ts_size"]])
        
        # Define main array for video to store.
        img_stack = np.zeros([0, par["frames"], par["img_size"], par["img_size"]])
        sig_stack = np.zeros([0, par["frames"], par["ts_size"]])

        # Use only selected building.
        if par["multiple_buildings"] == False:
            if int(building)  != par["selected_building"]:
                print_log(par,"skipping building "f"{building} due to parameter multiple_buildings ")
                continue
        
        # Filter out labels (appliances) with appliance.
        for meter in dataset.buildings[building].elec.submeters().meters:  
            
            # Get appliance name.
            label = meter.appliances[0].metadata.get("type")
            
            # Continue only for appliance from the main loop.
            if label != appliance : continue 

            # Load data into RAM.
            signal,time_stamps = mount_data(meter, par)

            # Slice time stamps and signal data to specified length.
            time_stamps_slices = moving_window(time_stamps, par["ts_size"])
            signal_slices = moving_window(signal, par["ts_size"])

            # Filter out low entropy data. 
            signal_slices, time_stamps_slices = filter_empty_slices_and_fill_missing_samples(signal_slices, time_stamps_slices, par)
            signal_slices, time_stamps_slices = filter_low_entropy_slices(signal_slices, time_stamps_slices, print_parameters)
            
            print_log(par,"Finished pre-processing! transforming...")

            # Continue if no data.
            if signal_slices.shape[0] == 0: continue
        
            # Define metrics.
            last_stamp = 0 # Used in append_images for calculating time delta.
            next_percent = 10 # Variable helps reduce log output in print_progress().

            # Transform pre-processed signal slices. 
            for i, [sig, time_stamps] in enumerate(zip(signal_slices, time_stamps_slices)):
    
                next_percent = print_progress(i, signal_slices, img_stack, next_percent, par)

                # Stop if enough data.
                if img_stack.shape[0] >= par["max_images"]:
                    print_log(par,"max size of "f"{par['max_images']} reached, skipping!")
                    break
                
                sig, img = transform_ts(sig, par)
                    
                # Append transformed images to stack.
                img_stack, img_stack_tmp, sig_stack, sig_stack_tmp, last_stamp = append_images(img, img_stack, img_stack_tmp,
                                                                                               sig, sig_stack, sig_stack_tmp,
                                                                                               time_stamps, last_stamp, par) 
        
        if img_stack.shape[0] > 0:
            # Save images.
            group_path = f"{par['dataset_name']}/"f"{appliance}/"f"{building}"
            store_many_hdf5(file_name, img_stack[...,np.newaxis], group_path, "img", force_del="yes")
            
            # Update metrics.
            images_stacked_per_appliance += img_stack.shape[0]
            images_stacked += img_stack.shape[0]
            healthy_appliances.add(appliance)
            
            if par["ts_save"]:
                # Save source time series.
                store_many_hdf5(file_name, sig_stack, group_path, "ts", force_del="yes")
        
        else:
            print_log(par,"empty for building", building, "appliance", appliance)
  
        print_log(par, "finished building N", building)
    
    print_end_of_loop(images_stacked_per_appliance, appliance, par)
    
print_end(images_stacked, healthy_appliances, par)

dataset name:  iawe , date :  2022-03-07 14:52:24  
 using appliances:  ['washing machine', 'television', 'air conditioner', 'motor', 'computer', 'wet appliance', 'fridge', 'unknown', 'clothes iron'] 
 transtype:  GAF 
 gaf trans type: GASF 
 windows size in mins: 13 
 image size in pixels: 100 
 sample period: 6 
 number if images that are stacked together and in series(series!): 5 
 allowed max delta between images  1800 
 added brightness:  True 
 save source timeseries: False 
 manually select appliances: False 
 number of appliances: 9 
 number of buildings: 1 
 max number of images per appliance per building: 10 
 include multiple buildings (Y for Yes N for No): True 
 building selected: A 

creating new file! ...

 Starting washing machine (1/9):

 Starting building 1
Finished pre-processing! transforming...


NameError: name 'print_flag' is not defined

In [28]:

try:
    a = 1.0/0

except ValueError:
    a = 0

except ZeroDivisionError:
    a = 0

print(a)

    

0
